In [1]:
import os
import re
import traceback
from typing import List, Tuple, Union, Dict, Any
import time
import torch
from transformers.utils import logging
from fastcore.xtras import Path

/Users/sbrewer/anaconda3/envs/TTS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path=(Path('.').cwd()/'microsoft'/'VibeVoice')
path.exists()

True

In [3]:
import sys 
sys.path.append(str(path))

In [4]:
from vibevoice.modular.modeling_vibevoice_inference import VibeVoiceForConditionalGenerationInference
from vibevoice.processor.vibevoice_processor import VibeVoiceProcessor

APEX FusedRMSNorm not available, using native implementation


In [5]:
class VoiceMapper:
    """Maps speaker names to voice file paths"""
    
    def __init__(self):
        self.setup_voice_presets()

        # change name according to our preset wav file
        new_dict = {}
        for name, path in self.voice_presets.items():
            
            if '_' in name:
                name = name.split('_')[0]
            
            if '-' in name:
                name = name.split('-')[-1]

            new_dict[name] = path
        self.voice_presets.update(new_dict)
        # print(list(self.voice_presets.keys()))

    def setup_voice_presets(self):
        """Setup voice presets by scanning the voices directory."""
        voices_dir =str(path/'demo'/'voices')
        #voices_dir = os.path.join(os.path.dirname(__file__), "voices")
        
        # Check if voices directory exists
        if not os.path.exists(voices_dir):
            print(f"Warning: Voices directory not found at {voices_dir}")
            self.voice_presets = {}
            self.available_voices = {}
            return
        
        # Scan for all WAV files in the voices directory
        self.voice_presets = {}
        
        # Get all .wav files in the voices directory
        wav_files = [f for f in os.listdir(voices_dir) 
                    if f.lower().endswith('.wav') and os.path.isfile(os.path.join(voices_dir, f))]
        
        # Create dictionary with filename (without extension) as key
        for wav_file in wav_files:
            # Remove .wav extension to get the name
            name = os.path.splitext(wav_file)[0]
            # Create full path
            full_path = os.path.join(voices_dir, wav_file)
            self.voice_presets[name] = full_path
        
        # Sort the voice presets alphabetically by name for better UI
        self.voice_presets = dict(sorted(self.voice_presets.items()))
        
        # Filter out voices that don't exist (this is now redundant but kept for safety)
        self.available_voices = {
            name: path for name, path in self.voice_presets.items()
            if os.path.exists(path)
        }
        
        print(f"Found {len(self.available_voices)} voice files in {voices_dir}")
        print(f"Available voices: {', '.join(self.available_voices.keys())}")

    def get_voice_path(self, speaker_name: str) -> str:
        """Get voice file path for a given speaker name"""
        # First try exact match
        if speaker_name in self.voice_presets:
            return self.voice_presets[speaker_name]
        
        # Try partial matching (case insensitive)
        speaker_lower = speaker_name.lower()
        for preset_name, path in self.voice_presets.items():
            if preset_name.lower() in speaker_lower or speaker_lower in preset_name.lower():
                return path
        
        # Default to first voice if no match found
        default_voice = list(self.voice_presets.values())[0]
        print(f"Warning: No voice preset found for '{speaker_name}', using default voice: {default_voice}")
        return default_voice

In [6]:
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
device

'mps'

In [7]:
# Initialize voice mapper
voice_mapper = VoiceMapper()

Found 9 voice files in /Users/sbrewer/Documents/LDM/microsoft/VibeVoice/demo/voices
Available voices: en-Alice_woman, en-Carter_man, en-Frank_man, en-Mary_woman_bgm, en-Maya_woman, in-Samuel_man, zh-Anchen_man_bgm, zh-Bowen_man, zh-Xinran_woman


In [8]:
txt_content = """"
Speaker 1: Drones, you are clear to assume autonomous operations. Good luck. Ground Control out.
Speaker 2: Drone 1, copy. Identifying hostile targets. 
Speaker 3: Drone 2, copy. Tally 3 tanks in the open, engaging with strike maneuver. 
Speaker 2: Drone 1, copy. I have your back Drone 2. Strike maneuver engaged.
Speaker 3: Drone 2, copy. Missiles away.
Speaker 1: Drones, this is ground control, confirm BDA. 
Speaker 3: Drone 2, copy. splash made, threat neutralized. 

"""

In [9]:
def parse_txt_script(txt_content: str) -> Tuple[List[str], List[str]]:
    """
    Parse txt script content and extract speakers and their text
    Fixed pattern: Speaker 1, Speaker 2, Speaker 3, Speaker 4
    Returns: (scripts, speaker_numbers)
    """
    lines = txt_content.strip().split('\n')
    scripts = []
    speaker_numbers = []
    
    # Pattern to match "Speaker X:" format where X is a number
    speaker_pattern = r'^Speaker\s+(\d+):\s*(.*)$'
    
    current_speaker = None
    current_text = ""
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        match = re.match(speaker_pattern, line, re.IGNORECASE)
        if match:
            # If we have accumulated text from previous speaker, save it
            if current_speaker and current_text:
                scripts.append(f"Speaker {current_speaker}: {current_text.strip()}")
                speaker_numbers.append(current_speaker)
            
            # Start new speaker
            current_speaker = match.group(1).strip()
            current_text = match.group(2).strip()
        else:
            # Continue text for current speaker
            if current_text:
                current_text += " " + line
            else:
                current_text = line
    
    # Don't forget the last speaker
    if current_speaker and current_text:
        scripts.append(f"Speaker {current_speaker}: {current_text.strip()}")
        speaker_numbers.append(current_speaker)
    
    return scripts, speaker_numbers

In [10]:
scripts, speaker_numbers = parse_txt_script(txt_content)

In [11]:
print(f"Found {len(scripts)} speaker segments:")
for i, (script, speaker_num) in enumerate(zip(scripts, speaker_numbers)):
    print(f"  {i+1}. Speaker {speaker_num}")
    print(f"     Text preview: {script[:100]}...")

Found 7 speaker segments:
  1. Speaker 1
     Text preview: Speaker 1: Drones, you are clear to assume autonomous operations. Good luck. Ground Control out....
  2. Speaker 2
     Text preview: Speaker 2: Drone 1, copy. Identifying hostile targets....
  3. Speaker 3
     Text preview: Speaker 3: Drone 2, copy. Tally 3 tanks in the open, engaging with strike maneuver....
  4. Speaker 2
     Text preview: Speaker 2: Drone 1, copy. I have your back Drone 2. Strike maneuver engaged....
  5. Speaker 3
     Text preview: Speaker 3: Drone 2, copy. Missiles away....
  6. Speaker 1
     Text preview: Speaker 1: Drones, this is ground control, confirm BDA....
  7. Speaker 3
     Text preview: Speaker 3: Drone 2, copy. splash made, threat neutralized....


In [12]:
speaker_numbers

['1', '2', '3', '2', '3', '1', '3']

In [13]:
speaker_names_list = ['Alice', 'Frank', 'Carter', 'Mary', 'Maya']
speaker_name_mapping= {str(i): name for i, name in enumerate(speaker_names_list, 1)}
speaker_name_mapping

{'1': 'Alice', '2': 'Frank', '3': 'Carter', '4': 'Mary', '5': 'Maya'}

In [14]:
print(f"\nSpeaker mapping:")
for speaker_num in set(speaker_numbers):
    mapped_name = speaker_name_mapping.get(speaker_num, f"Speaker {speaker_num}")
    print(f"  Speaker {speaker_num} -> {mapped_name}")


Speaker mapping:
  Speaker 1 -> Alice
  Speaker 2 -> Frank
  Speaker 3 -> Carter


In [15]:
# Map speakers to voice files using the provided speaker names
voice_samples = []
actual_speakers = []

# Get unique speaker numbers in order of first appearance
unique_speaker_numbers = []
seen = set()
for speaker_num in speaker_numbers:
    if speaker_num not in seen:
        unique_speaker_numbers.append(speaker_num)
        seen.add(speaker_num)

In [16]:
for speaker_num in unique_speaker_numbers:
    speaker_name = speaker_name_mapping.get(speaker_num, f"Speaker {speaker_num}")
    voice_path = voice_mapper.get_voice_path(speaker_name)
    voice_samples.append(voice_path)
    actual_speakers.append(speaker_name)
    print(f"Speaker {speaker_num} ('{speaker_name}') -> Voice: {os.path.basename(voice_path)}")

Speaker 1 ('Alice') -> Voice: en-Alice_woman.wav
Speaker 2 ('Frank') -> Voice: en-Frank_man.wav
Speaker 3 ('Carter') -> Voice: en-Carter_man.wav


In [17]:
# Prepare data for model
full_script = '\n'.join(scripts)
full_script = full_script.replace("’", "'")   

In [18]:
print(full_script)

Speaker 1: Drones, you are clear to assume autonomous operations. Good luck. Ground Control out.
Speaker 2: Drone 1, copy. Identifying hostile targets.
Speaker 3: Drone 2, copy. Tally 3 tanks in the open, engaging with strike maneuver.
Speaker 2: Drone 1, copy. I have your back Drone 2. Strike maneuver engaged.
Speaker 3: Drone 2, copy. Missiles away.
Speaker 1: Drones, this is ground control, confirm BDA.
Speaker 3: Drone 2, copy. splash made, threat neutralized.


In [19]:
model_path = str(path.parent/'VibeVoice-1pt5B')
processor = VibeVoiceProcessor.from_pretrained(model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'VibeVoiceTextTokenizerFast'.


In [20]:
(path.parent/'VibeVoice-1pt5B').exists()

True

In [21]:
# Decide dtype & attention implementation
if device == "mps":
    load_dtype = torch.float32  # MPS requires float32
    attn_impl_primary = "sdpa"  # flash_attention_2 not supported on MPS
elif device == "cuda":
    load_dtype = torch.bfloat16
    attn_impl_primary = "flash_attention_2"
else:  # cpu
    load_dtype = torch.float32
    attn_impl_primary = "sdpa"
print(f"Using device: {device}, torch_dtype: {load_dtype}, attn_implementation: {attn_impl_primary}")

Using device: mps, torch_dtype: torch.float32, attn_implementation: sdpa


In [22]:
model = VibeVoiceForConditionalGenerationInference.from_pretrained(
                model_path,
                torch_dtype=load_dtype,
                attn_implementation=attn_impl_primary,
                device_map=None,  # load then move
            )
model.to(device)

Loading checkpoint shards: 100%|██████████████████| 3/3 [00:02<00:00,  1.17it/s]


VibeVoiceForConditionalGenerationInference(
  (model): VibeVoiceModel(
    (language_model): Qwen2Model(
      (embed_tokens): Embedding(151936, 1536)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
            (k_proj): Linear(in_features=1536, out_features=256, bias=True)
            (v_proj): Linear(in_features=1536, out_features=256, bias=True)
            (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
            (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
            (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((

In [23]:
model.eval()
model.set_ddpm_inference_steps(num_steps=10)

In [24]:
if hasattr(model.model, 'language_model'):
    print(f"Language model attention: {model.model.language_model.config._attn_implementation}")

Language model attention: sdpa


In [25]:
# Prepare inputs for the model
inputs = processor(
        text=[full_script], # Wrap in list for batch processing
        voice_samples=[voice_samples], # Wrap in list for batch processing
        padding=True,
        return_tensors="pt",
        return_attention_mask=True,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# Move tensors to target device
for k, v in inputs.items():
    if torch.is_tensor(v):
        inputs[k] = v.to(device)

In [27]:
cfg_scale = 1.3
print(f"Starting generation with cfg_scale: {cfg_scale}")

Starting generation with cfg_scale: 1.3


In [28]:
# Generate audio
start_time = time.time()
outputs = model.generate(
        **inputs,
        max_new_tokens=None,
        cfg_scale=cfg_scale,
        tokenizer=processor.tokenizer,
        generation_config={'do_sample': False},
        verbose=True,
    )
generation_time = time.time() - start_time
print(f"Generation time: {generation_time:.2f} seconds")

Generating (active: 1/1):  21%|██▉           | 264/1280 [01:20<05:28,  3.09it/s]

Samples [0] reached EOS token at step 266.


Generation time: 80.60 seconds


In [29]:
sample_rate = 24000
audio_samples = outputs.speech_outputs[0].shape[-1] if len(outputs.speech_outputs[0].shape) > 0 else len(outputs.speech_outputs[0])
audio_duration = audio_samples / sample_rate
rtf = generation_time / audio_duration if audio_duration > 0 else float('inf')

print(f"Generated audio duration: {audio_duration:.2f} seconds")
print(f"RTF (Real Time Factor): {rtf:.2f}x")

Generated audio duration: 33.60 seconds
RTF (Real Time Factor): 2.40x


In [30]:
# Calculate token metrics
input_tokens = inputs['input_ids'].shape[1]  # Number of input tokens
output_tokens = outputs.sequences.shape[1]  # Total tokens (input + generated)
generated_tokens = output_tokens - input_tokens

print(f"Prefilling tokens: {input_tokens}")
print(f"Generated tokens: {generated_tokens}")
print(f"Total tokens: {output_tokens}")

Prefilling tokens: 640
Generated tokens: 266
Total tokens: 906


In [31]:
processor.save_audio(
        outputs.speech_outputs[0], # First (and only) batch item
        output_path='test.wav',
    )

['test.wav']

In [32]:
from IPython.display import Audio, display
display(Audio('test.wav'))

In [33]:
for i in unique_speaker_numbers:
    txt_content=txt_content.replace(f'Speaker {i}', speaker_name_mapping.get(str(i)))

print(txt_content)

"
Alice: Drones, you are clear to assume autonomous operations. Good luck. Ground Control out.
Frank: Drone 1, copy. Identifying hostile targets. 
Carter: Drone 2, copy. Tally 3 tanks in the open, engaging with strike maneuver. 
Frank: Drone 1, copy. I have your back Drone 2. Strike maneuver engaged.
Carter: Drone 2, copy. Missiles away.
Alice: Drones, this is ground control, confirm BDA. 
Carter: Drone 2, copy. splash made, threat neutralized. 


